<a href="https://colab.research.google.com/github/NPCA-TEAM/COVID-19/blob/main/Scripts/AJUSTE_DA_PROJE%C3%87%C3%83O_POS_USO_MODELO_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparação preliminar**

In [ ]:
#Inicia permitindo acesso ao Gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Instalação das bibliotecas

In [ ]:
!pip install pyyaml==5.4.1

In [ ]:
!pip install darts

In [ ]:
!pip install matplotlib==3.6.2

In [ ]:
!pip install -U csaps

## Importação das bibliotecas

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import glob
import os
import sys
import time
import json

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

from datetime import datetime
from darts import timeseries
from darts.dataprocessing.transformers import Scaler
from darts import TimeSeries

from darts.models import (    
    NBEATSModel,
    TCNModel,
    TransformerModel,
    TFTModel,
    NHiTSModel
)

from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.models import KalmanFilter
from csaps import csaps

import warnings
warnings.filterwarnings('ignore')
import logging
logging.disable(logging.CRITICAL)
np.random.seed(1)  # for reproducibility

# **Importação dos dados**

## Dados e campos de entrada

In [ ]:
# Caminhos dos datasets
path_fittedforecast_casos = '/content/drive/MyDrive/NPCA - COVID/_CASOS/FittedForecastsPosSelectedModel/'
path_fittedforecast_obitos = '/content/drive/MyDrive/NPCA - COVID/_OBITOS/FittedForecastsPosSelectedModel/'

path_limits_casos = '/content/drive/MyDrive/NPCA - COVID/_CASOS/ResultsOfSelectedModel/MarginOfError/'
path_limits_obitos = '/content/drive/MyDrive/NPCA - COVID/_OBITOS/ResultsOfSelectedModel/MarginOfError/'

#Pasta para salvar dados gerados 
path_figs_casos = '/content/drive/MyDrive/NPCA - COVID/_CASOS/Figs/'
path_figs_obitos = '/content/drive/MyDrive/NPCA - COVID/_OBITOS/Figs/'

path_csv_casos = '/content/drive/MyDrive/NPCA - COVID/_CASOS/ForecastsPredict/'
path_csv_obitos = '/content/drive/MyDrive/NPCA - COVID/_OBITOS/ForecastsPredict/'

## Leitura do dataset e seleção das variáveis

In [ ]:
# Obtenção e ordenação de lisa de dataframes para casos
file_list_dataframe_casos = glob.glob(f'{path_fittedforecast_casos}*.xlsx')
sorted_list_dataframe_casos = sorted(file_list_dataframe_casos, key=os.path.getctime, reverse = True)

# Filtro para seleção do arquivo apropriado
dataframe_casos_publicacao = pd.read_excel(list(filter(lambda x: 'publicacao' in x, sorted_list_dataframe_casos))[0])
dataframe_casos_sintomas = pd.read_excel(list(filter(lambda x: 'sintomas' in x, sorted_list_dataframe_casos))[0])


# Obtenção e ordenação de lisa de dataframes para a margem de erro de casos
file_list_limits_casos = glob.glob(f'{path_limits_casos}*.csv')
sorted_list_limits_casos = sorted(file_list_limits_casos, key=os.path.getctime, reverse = True)

# Filtro para seleção do arquivo apropriado
limites_casos_publicacao = pd.read_csv(list(filter(lambda x: 'publicacao' in x, sorted_list_limits_casos))[0])
limites_casos_sintomas = pd.read_csv(list(filter(lambda x: 'sintomas' in x, sorted_list_limits_casos))[0])

In [ ]:
# Obtenção e ordenação de lisa de dataframes para obitos
file_list_dataframe_obitos = glob.glob(f'{path_fittedforecast_obitos}*.xlsx')
sorted_list_dataframe_obitos = sorted(file_list_dataframe_obitos, key=os.path.getctime, reverse = True)

# Filtro para seleção do arquivo apropriado
dataframe_obitos_publicacao = pd.read_excel(list(filter(lambda x: 'publicacao' in x, sorted_list_dataframe_obitos))[0])
dataframe_obitos_ocorrencia = pd.read_excel(list(filter(lambda x: 'ocorrencia' in x, sorted_list_dataframe_obitos))[0])


# Obtenção e ordenação de lisa de dataframes para a margem de erro de obitos
file_list_limits_obitos = glob.glob(f'{path_limits_obitos}*.csv')
sorted_list_limits_obitos = sorted(file_list_limits_obitos, key=os.path.getctime, reverse = True)

# Filtro para seleção do arquivo apropriado
limites_obitos_publicacao = pd.read_csv(list(filter(lambda x: 'publicacao' in x, sorted_list_limits_obitos))[0])
limites_obitos_ocorrencia = pd.read_csv(list(filter(lambda x: 'ocorrencia' in x, sorted_list_limits_obitos))[0])

In [ ]:
# Seleção das variáveis alvo
casos_publicacao = dataframe_casos_publicacao.loc[:, ['day', 'obs','sim','fitted']]
casos_sintomas = dataframe_casos_sintomas.loc[:, ['day', 'obs','sim','fitted']]

obitos_publicacao = dataframe_obitos_publicacao.loc[:, ['day', 'obs','sim','fitted']]
obitos_ocorrencia = dataframe_obitos_ocorrencia.loc[:, ['day', 'obs','sim','fitted']]


# Renomeaçõa da variavel de tempo
casos_publicacao.rename(columns = {'day':'data'}, inplace = True)
casos_sintomas.rename(columns = {'day':'data'}, inplace = True)

obitos_publicacao.rename(columns = {'day':'data'}, inplace = True)
obitos_ocorrencia.rename(columns = {'day':'data'}, inplace = True)

In [ ]:
obitos_publicacao

# **Pre-processamento dos dados**

## Instanciação das séries temporais

In [ ]:
# criação das séries temporais alvos de casos
serie_casos_publicacao = timeseries.TimeSeries.from_dataframe(df=casos_publicacao, time_col='data')
serie_casos_sintomas = timeseries.TimeSeries.from_dataframe(df=casos_sintomas, time_col='data')

serie_casos_publicacao.plot()
serie_casos_sintomas.plot()

In [ ]:
# criação das séries temporais alvos de obitos
serie_obitos_publicacao = timeseries.TimeSeries.from_dataframe(df=obitos_publicacao, time_col='data')
serie_obitos_ocorrencia = timeseries.TimeSeries.from_dataframe(df=obitos_ocorrencia, time_col='data')

serie_obitos_publicacao.plot()
#serie_obitos_ocorrencia.plot()

In [ ]:
# Criação das series temporais das margens de erro de casos
serie_limites_casos_publicacao = timeseries.TimeSeries.from_dataframe(df=limites_casos_publicacao, time_col='data')
serie_limites_casos_sintomas = timeseries.TimeSeries.from_dataframe(df=limites_casos_sintomas, time_col='data')

#serie_limites_casos_publicacao.plot()
#serie_casos_publicacao['sim'].plot()

serie_limites_casos_sintomas.plot()
serie_casos_sintomas['sim'].plot()

In [ ]:
# Criação das series temporais das margens de erro de obitos
serie_limites_obitos_publicacao = timeseries.TimeSeries.from_dataframe(df=limites_obitos_publicacao, time_col='data')
serie_limites_obitos_ocorrencia = timeseries.TimeSeries.from_dataframe(df=limites_obitos_ocorrencia, time_col='data')

#serie_limites_obitos_publicacao.plot()
#serie_obitos_publicacao['sim'].plot()

serie_limites_obitos_ocorrencia.plot()
serie_obitos_ocorrencia['sim'].plot()

## Filtro de suavisação

In [ ]:
def smoothing_splines_filter(serie):
  y_serieValues=serie.values()
  ind_final = len(y_serieValues)
  y_serieValues = y_serieValues.flatten().copy()
  x_serieValues = np.array(range(0,ind_final))

  yi_serieValues, smooth_serieValues = csaps(x_serieValues, y_serieValues, x_serieValues)
  
  time_idx = serie.time_index
  smooth_serie = TimeSeries.from_times_and_values(time_idx, yi_serieValues,columns=serie.columns)
  return smooth_serie

In [ ]:
casos_publicacao_limitsup_suav = smoothing_splines_filter(serie_limites_casos_publicacao['Casos_Publicacao_Max'])
casos_publicacao_limitinf_suav = smoothing_splines_filter(serie_limites_casos_publicacao['Casos_Publicacao_Min'])

In [ ]:
casos_sintomas_limitsup_suav = smoothing_splines_filter(serie_limites_casos_sintomas['Casos_DataSintoma_Max'])
casos_sintomas_limitinf_suav = smoothing_splines_filter(serie_limites_casos_sintomas['Casos_DataSintoma_Min'])

In [ ]:
obitos_publicacao_limitsup_suav = smoothing_splines_filter(serie_limites_obitos_publicacao['Obitos_Publicacao_Max'])
obitos_publicacao_limitinf_suav = smoothing_splines_filter(serie_limites_obitos_publicacao['Obitos_Publicacao_Min'])

In [ ]:
obitos_ocorrencia_limitsup_suav = smoothing_splines_filter(serie_limites_obitos_ocorrencia['Obitos_DataOcorrencia_Max'])
obitos_ocorrencia_limitinf_suav = smoothing_splines_filter(serie_limites_obitos_ocorrencia['Obitos_DataOcorrencia_Min'])

##Gerar novos limites

In [ ]:
def calculate_new_limits(limsup, liminf, proj, newproj): 
  # Mover a margem de erro em proporção ao projetado
  newlimitsup = abs(limsup - proj) + newproj
  newlimitinf = (abs(liminf - proj) - newproj)*-1

  return newlimitsup, newlimitinf

In [ ]:
# Ajuste da margem de erro para casos
casos_publicacao_limitsup, casos_publicacao_limitinf = calculate_new_limits(casos_publicacao_limitsup_suav,
                                                                            casos_publicacao_limitinf_suav,
                                                                            serie_casos_publicacao['sim'],
                                                                            serie_casos_publicacao['fitted'])


casos_sintomas_limitsup, casos_sintomas_limitinf = calculate_new_limits(casos_sintomas_limitsup_suav, 
                                                                        casos_sintomas_limitinf_suav, 
                                                                        serie_casos_sintomas['sim'], 
                                                                        serie_casos_sintomas['fitted'])


#casos_publicacao_limitsup.plot()
#casos_publicacao_limitinf.plot()
#serie_casos_publicacao['fitted'].plot()

#casos_sintomas_limitsup.plot()
#casos_sintomas_limitinf.plot()
#serie_casos_sintomas['fitted'].plot()

In [ ]:
# Ajuste da margem de erro para obitos
obitos_publicacao_limitsup, obitos_publicacao_limitinf = calculate_new_limits(obitos_publicacao_limitsup_suav,
                                                                              obitos_publicacao_limitinf_suav,
                                                                              serie_obitos_publicacao['sim'],
                                                                              serie_obitos_publicacao['fitted'])


obitos_ocorrencia_limitsup, obitos_ocorrencia_limitinf = calculate_new_limits(obitos_ocorrencia_limitsup_suav,
                                                                              obitos_ocorrencia_limitinf_suav,
                                                                              serie_obitos_ocorrencia['sim'],
                                                                              serie_obitos_ocorrencia['fitted'])


#obitos_publicacao_limitsup.plot()
#obitos_publicacao_limitinf.plot()
#serie_obitos_publicacao['fitted'].plot()

#obitos_ocorrencia_limitsup.plot()
#obitos_ocorrencia_limitinf.plot()
#serie_obitos_ocorrencia['fitted'].plot()

# **Exportação e visualização**

## Preparação para plotagem e exportação

### Com os limites

In [ ]:
def plot_preparation (serie, limitsup, limitinf, real, serie_name):
  # Prearação da serie
  serie_real = real.pd_dataframe()
  serie_median = serie.pd_dataframe()
  serie_max = limitsup.pd_dataframe()
  serie_min = limitinf.pd_dataframe()
  
  serie_real['obs'] = np.where(serie_real['obs'] < 0, 0, serie_real['obs'])
  serie_median['fitted'] = np.where(serie_median['fitted'] < 0, 0, serie_median['fitted'])
  serie_max[f'{serie_name}_Max'] = np.where(serie_max[f'{serie_name}_Max'] < 0, 0, serie_max[f'{serie_name}_Max'])
  serie_min[f'{serie_name}_Min'] = np.where(serie_min[f'{serie_name}_Min'] < 0, 0, serie_min[f'{serie_name}_Min'])

  serie_real.rename(columns = {'obs':'Observed'}, inplace=True)
  serie_median.rename(columns = {'fitted':'Simulated'}, inplace=True)
  serie_max.rename(columns = {f'{serie_name}_Max':'Superior Margin'}, inplace = True)
  serie_min.rename(columns = {f'{serie_name}_Min':'Inferior Margin'}, inplace = True)

  return serie_real, serie_median, serie_max, serie_min

In [ ]:
#Preparação dos dados para plotagem
casos_publicacao_real, casos_publicacao_median, casos_publicacao_max, casos_publicacao_min = plot_preparation(serie_casos_publicacao['fitted'],
                                                                                                              casos_publicacao_limitsup, casos_publicacao_limitinf,
                                                                                                              serie_casos_publicacao['obs'], 'Casos_Publicacao')

In [ ]:
#Preparação dos dados para plotagem
casos_sintomas_real, casos_sintomas_median, casos_sintomas_max, casos_sintomas_min = plot_preparation(serie_casos_sintomas['fitted'],
                                                                                                      casos_sintomas_limitsup, casos_sintomas_limitinf,
                                                                                                      serie_casos_sintomas['obs'], 'Casos_DataSintoma')

In [ ]:
#Preparação dos dados para plotagem
obitos_publicacao_real, obitos_publicacao_median, obitos_publicacao_max, obitos_publicacao_min = plot_preparation(serie_obitos_publicacao['fitted'],
                                                                                                                  obitos_publicacao_limitsup, obitos_publicacao_limitinf,
                                                                                                                  serie_obitos_publicacao['obs'], 'Obitos_Publicacao')

In [ ]:
#Preparação dos dados para plotagem
obitos_ocorrencia_real, obitos_ocorrencia_median, obitos_ocorrencia_max, obitos_ocorrencia_min = plot_preparation(serie_obitos_ocorrencia['fitted'],
                                                                                                                  obitos_ocorrencia_limitsup, obitos_ocorrencia_limitinf,
                                                                                                                  serie_obitos_ocorrencia['obs'], 'Obitos_DataOcorrencia')

### Sem os limites

In [ ]:
def plot_preparation (serie, real, serie_name):
  # Prearação da serie
  serie_real = real.pd_dataframe()
  serie_median = serie.pd_dataframe()
  
  serie_real['obs'] = np.where(serie_real['obs'] < 0, 0, serie_real['obs'])
  serie_median['fitted'] = np.where(serie_median['fitted'] < 0, 0, serie_median['fitted'])

  serie_real.rename(columns = {'obs':'Observed'}, inplace=True)
  serie_median.rename(columns = {'fitted':'Simulated'}, inplace=True)

  return serie_real, serie_median

In [ ]:
#Preparação dos dados para plotagem
casos_publicacao_real, casos_publicacao_median = plot_preparation(serie_casos_publicacao['fitted'], serie_casos_publicacao['obs'], 'Casos_Publicacao')

In [ ]:
#Preparação dos dados para plotagem
casos_sintomas_real, casos_sintomas_median = plot_preparation(serie_casos_sintomas['fitted'], serie_casos_sintomas['obs'], 'Casos_DataSintoma')

In [ ]:
#Preparação dos dados para plotagem
obitos_publicacao_real, obitos_publicacao_median = plot_preparation(serie_obitos_publicacao['fitted'], serie_obitos_publicacao['obs'], 'Obitos_Publicacao')

In [ ]:
#Preparação dos dados para plotagem
obitos_ocorrencia_real, obitos_ocorrencia_median = plot_preparation(serie_obitos_ocorrencia['fitted'], serie_obitos_ocorrencia['obs'], 'Obitos_DataOcorrencia')

##Exportação dos dataframes com as predições

In [ ]:
experiment_date = input('Data do experimento: ')

ModeloName_casos_publicacao = input('Nome do modelo para casos publicaçao: ')
ModeloName_casos_sintomas = input('Nome do modelo para casos sintomas: ')

ModeloName_obitos_publicacao = input('Nome do modelo para obitos publicaçao: ')
ModeloName_obitos_ocorrencia = input('Nome do modelo para obitos ocorrencia: ')

### Com os limites

In [ ]:
# Exportação das previsões para casos
casos_publicacao_toExport = pd.concat([casos_publicacao_real, casos_publicacao_median, casos_publicacao_min, casos_publicacao_max], axis=1)
casos_sintomas_toExport = pd.concat([casos_sintomas_real, casos_sintomas_median, casos_sintomas_min, casos_sintomas_max], axis=1)

casos_publicacao_toExport.index.name = 'Date'
casos_sintomas_toExport.index.name = 'Date'

casos_publicacao_toExport.to_csv(path_or_buf = f'{path_fittedforecast_casos}/casos_publicacao_fittedforecast_{ModeloName_casos_publicacao}_{experiment_date}.csv')
casos_sintomas_toExport.to_csv(path_or_buf = f'{path_fittedforecast_casos}/casos_sintomas_fittedforecast_{ModeloName_casos_sintomas}_{experiment_date}.csv')

In [ ]:
# Exportação das previsões para casos
obitos_publicacao_toExport = pd.concat([obitos_publicacao_real, obitos_publicacao_median, obitos_publicacao_min, obitos_publicacao_max], axis=1)
obitos_ocorrencia_toExport = pd.concat([obitos_ocorrencia_real, obitos_ocorrencia_median, obitos_ocorrencia_min, obitos_ocorrencia_max], axis=1)

obitos_publicacao_toExport.index.name = 'Date'
obitos_ocorrencia_toExport.index.name = 'Date'

obitos_publicacao_toExport.to_csv(path_or_buf = f'{path_fittedforecast_obitos}/obitos_publicacao_fittedforecast_{ModeloName_obitos_publicacao}_{experiment_date}.csv')
obitos_ocorrencia_toExport.to_csv(path_or_buf = f'{path_fittedforecast_obitos}/obitos_ocorrencia_fittedforecast_{ModeloName_obitos_ocorrencia}_{experiment_date}.csv')

### Sem os limites

In [ ]:
# Exportação das previsões para casos
casos_publicacao_toExport = pd.concat([casos_publicacao_real, casos_publicacao_median], axis=1)
casos_sintomas_toExport = pd.concat([casos_sintomas_real, casos_sintomas_median], axis=1)

casos_publicacao_toExport.index.name = 'Date'
casos_sintomas_toExport.index.name = 'Date'

casos_publicacao_toExport.to_csv(path_or_buf = f'{path_fittedforecast_casos}/casos_publicacao_fittedforecast_{ModeloName_casos_publicacao}_{experiment_date}.csv')
casos_sintomas_toExport.to_csv(path_or_buf = f'{path_fittedforecast_casos}/casos_sintomas_fittedforecast_{ModeloName_casos_sintomas}_{experiment_date}.csv')

In [ ]:
# Exportação das previsões para casos
obitos_publicacao_toExport = pd.concat([obitos_publicacao_real, obitos_publicacao_median], axis=1)
obitos_ocorrencia_toExport = pd.concat([obitos_ocorrencia_real, obitos_ocorrencia_median], axis=1)

obitos_publicacao_toExport.index.name = 'Date'
obitos_ocorrencia_toExport.index.name = 'Date'

obitos_publicacao_toExport.to_csv(path_or_buf = f'{path_fittedforecast_obitos}/obitos_publicacao_fittedforecast_{ModeloName_obitos_publicacao}_{experiment_date}.csv')
obitos_ocorrencia_toExport.to_csv(path_or_buf = f'{path_fittedforecast_obitos}/obitos_ocorrencia_fittedforecast_{ModeloName_obitos_ocorrencia}_{experiment_date}.csv')

##Visualização das projeções e exportação dos gráficos

In [ ]:
#Função responsavel pela plotagem geral
def plot(median, min, max, real, titulo, pasta, date):
  plt.style.use('seaborn-dark-palette')
  fig, ax = plt.subplots(figsize=(18, 6))


  ax.plot(real, lw=1.5, linestyle='solid', label='Observed') #Dados observados
  ax.plot(median[:-7], lw=1.5, linestyle='solid', label='Historical forecast') #Historical
  ax.plot(median[-8:], lw=1.5, linestyle='solid', label='Ahead forecast') #Predição
  ax.fill_between(median.index, np.array(min).flatten(), np.array(max).flatten(), alpha=.15, label='Margin of error') #Margem de erro
  ax.axvline(x=median.index[-8], alpha=.5, linestyle='--', color='black') #Linha vertical

  top = ax.spines['top']
  top.set_visible(True)
  bottom = ax.spines['bottom']
  bottom.set_visible(True)  
  right = ax.spines['right']
  right.set_visible(True)  
  left = ax.spines['left']
  left.set_visible(True)


  min, max = ax.get_ylim()
  ax.yaxis.set_ticks(np.arange(0, max, max/10))
  ax.xaxis.set_major_locator(mdates.DayLocator(interval=20)) #Intervalo entre as marcações de tempo
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d')) #Formatação das datas


  ax.legend(loc='upper left', bbox_to_anchor=(0, 1.09), ncol = 2, fancybox=True, edgecolor='Black', framealpha=1, frameon=True)
  ax.set(title=titulo) #Titulo
  ax.margins(0, tight=True)
  ax.grid(False)


  fig.autofmt_xdate(rotation=75) #Angulo das datas
  fig.tight_layout()
  
  
  fig.savefig(f'{pasta}Plots/{titulo} - {date}.png', dpi=300) #Salvar as imagens

In [ ]:
#Função responsavel pela plotagem dos ultimos 30 dias
def plot_30(median, min, max, real, titulo, pasta, date):
  #Criação de series com apenas os ultimos 30 dias
  median_30 = median.tail(30)
  min_30 = min.tail(30)
  max_30 = max.tail(30)
  real_30 = real.tail(30) #O original para no 23 pq os ultimos 7 são predição


  plt.style.use('seaborn-dark-palette')
  fig, ax = plt.subplots(figsize=(18, 6))


  ax.plot(real_30, lw=1.5, linestyle='solid', label='Observed') #Dados observados
  ax.plot(median_30[:-7], lw=1.5, linestyle='solid', label='Historical forecast') #Historical
  ax.plot(median_30[-8:], lw=1.5, linestyle='solid', label='Ahead forecast') #Predição
  ax.fill_between(median_30.index, np.array(min_30).flatten(), np.array(max_30).flatten(), color='darkgreen', alpha=.15, label='Margin of error') #Margem de erro
  ax.axvline(x=median_30.index[-8], alpha=.5, linestyle='--', color='black') #Linha vertical

  top = ax.spines['top']
  top.set_visible(True)
  bottom = ax.spines['bottom']
  bottom.set_visible(True)  
  right = ax.spines['right']
  right.set_visible(True)  
  left = ax.spines['left']
  left.set_visible(True)

  min, max = ax.get_ylim()
  ax.yaxis.set_ticks(np.arange(0, max, max/10))
  ax.xaxis.set_major_locator(mdates.DayLocator(interval=1)) #Intervalo entre as marcações de tempo
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d')) #Formatação das datas


  ax.legend(loc='upper left', bbox_to_anchor=(0, 1.09), ncol = 2, fancybox=True, edgecolor='Black', framealpha=1, frameon=True)
  ax.set(title=titulo) #Titulo
  ax.margins(0, tight=True)
  ax.grid(False, which = 'major')


  fig.autofmt_xdate(rotation=75) #Angulo das datas
  fig.tight_layout()


  fig.savefig(f'{pasta}Plots/{titulo}_{date}.png', dpi=300) #Salvar as imagens

In [ ]:
# Plot casos - publicação - total
titulo_caso_publicação = f"Forecast of cases by day of publication (moving average) - {ModeloName_casos_publicacao}"
plot(casos_publicacao_median, casos_publicacao_min, casos_publicacao_max, casos_publicacao_real, titulo_caso_publicação, path_fittedforecast_casos, experiment_date)

In [ ]:
# Plot casos - publicação - ultimos 30 dias
titulo_caso_publicação_30 = f"Forecast of cases by day of publication (moving average) - Last 30 days - {ModeloName_casos_publicacao}"
plot_30(casos_publicacao_median, casos_publicacao_min, casos_publicacao_max, casos_publicacao_real, titulo_caso_publicação_30, path_fittedforecast_casos, experiment_date)

In [ ]:
# Plot casos - sintomas - total
titulo_caso_sintomas = f"Forecast of cases by day of the first symptoms (moving average) - {ModeloName_casos_sintomas}"
plot(casos_sintomas_median, casos_sintomas_min, casos_sintomas_max, casos_sintomas_real, titulo_caso_sintomas, path_fittedforecast_casos, experiment_date)

In [ ]:
# Plot casos - sintomas - ultimos 30 dias
titulo_caso_sintomas_30 = f"Forecast of cases by day of the first symptoms (moving average) - Last 30 days - {ModeloName_casos_sintomas}"
plot_30(casos_sintomas_median, casos_sintomas_min, casos_sintomas_max, casos_sintomas_real, titulo_caso_sintomas_30, path_fittedforecast_casos, experiment_date)

In [ ]:
# Plot obitos - publicaçao - total
titulo_obito_publicação = f'Forecast of deaths by day of publication (moving average) - {ModeloName_obitos_publicacao}'
plot(obitos_publicacao_median, obitos_publicacao_min, obitos_publicacao_max, obitos_publicacao_real, titulo_obito_publicação, path_fittedforecast_obitos, experiment_date)

In [ ]:
# Plot obitos - publicaçao - ultimos 30 dias
titulo_obito_publicação_30 = f'Forecast of deaths by day of publication (moving average) - Last 30 days - {ModeloName_obitos_publicacao}'
plot_30(obitos_publicacao_median, obitos_publicacao_min, obitos_publicacao_max, obitos_publicacao_real, titulo_obito_publicação_30, path_fittedforecast_obitos, experiment_date)

In [ ]:
# Plot obitos - ocorrencia - total
titulo_obito_ocorrencias = f'Forecast of deaths by day of occurrence (moving average) - {ModeloName_obitos_ocorrencia}'
plot(obitos_ocorrencia_median, obitos_ocorrencia_min, obitos_ocorrencia_max, obitos_ocorrencia_real, titulo_obito_ocorrencias, path_fittedforecast_obitos, experiment_date)

In [ ]:
# Plot obitos - ocorrencia - ultimos 30 dias
titulo_obito_ocorrencias_30 = f'Forecast of deaths by day of occurrence (moving average) - Last 30 days - {ModeloName_obitos_ocorrencia}'
plot_30(obitos_ocorrencia_median, obitos_ocorrencia_min, obitos_ocorrencia_max, obitos_ocorrencia_real, titulo_obito_ocorrencias_30, path_fittedforecast_obitos, experiment_date)